In [24]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [25]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\BreannaFrankenberg\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [26]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [27]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [28]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Mars 2020 Rover Completes Its First Drive</div>

In [29]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Mars 2020 Rover Completes Its First Drive"

In [30]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'In a 10-plus-hour marathon, the rover steered, turned and drove in 3-foot (1-meter) increments over small ramps.'

### Featured Images

In [31]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [32]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [33]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [34]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [35]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

In [36]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [37]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

### Hemispheres

In [38]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [39]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
main_url = 'https://marshemispheres.com/'
html = browser.html
hemisphere_url_soup = soup(html, 'html.parser')

In [40]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.
divs = hemisphere_url_soup.find("div", class_='downloads')
link_div = hemisphere_url_soup.find_all("div", class_="description")

anchors = browser.links.find_by_partial_text('Hemisphere')
relative_urls = set([anchor['href'] for anchor in anchors])

hemispheres = {}

for relative_url in relative_urls:
    
    full_url = f'{relative_url}'
    browser.visit(full_url)
    
    html = browser.html
    urls_soup = soup(html, 'html.parser')
    
    hemi_title = browser.find_by_tag('h2').text
    print(f'--> title: {hemi_title}')
    
    downloads_div = urls_soup.find('div', class_='downloads')
    img_anchor = downloads_div.find('a', text=('Sample'))
    img_url = img_anchor['href']
    full_img_url = f'{main_url}{img_url}'
    print(f'--> url: {main_url}{img_url}')
    
    hemispheres = {
           'img_url': full_img_url,
           'title': hemi_title,
    }
    
    browser.back()
        
    hemisphere_image_urls.append(hemispheres)

--> title: Valles Marineris Hemisphere Enhanced
--> url: https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg
--> title: Schiaparelli Hemisphere Enhanced
--> url: https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
--> title: Cerberus Hemisphere Enhanced
--> url: https://marshemispheres.com/images/full.jpg
--> title: Syrtis Major Hemisphere Enhanced
--> url: https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg


In [41]:
# 4. Print the list that holds the dictionary of each image url and title.
print(hemisphere_image_urls)

[{'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}]


In [42]:
# 5. Quit the browser
browser.quit()